<table class="table table-bordered">
    <tr>
        <th style="text-align:center; width:25%"><img src='https://www.np.edu.sg/PublishingImages/Pages/default/odp/ICT.jpg' style="width: 250px; height: 125px; "></th>
        <th style="text-align:center;"><h1>Deep Learning</h1><h2>Practical 9a - Keras Functional API</h2><h3>AY2020/21 Semester</h3></th>
    </tr>
</table>

### 1. Introduction to the functional API
In the functional API, you directly manipulate tensors, and you use layers as functions
that take tensors and return tensors (hence, the name functional API):

In [ ]:
from tensorflow.keras import Input, layers
input_tensor = Input(shape=(32,))

In [ ]:
dense = layers.Dense(32, activation='relu')
output_tensor = dense(input_tensor)

Let’s start with a minimal example that shows side by side a simple Sequential model
and its equivalent in the functional API:


In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras import Input

* ***A Sequential Model***

In [ ]:
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))
seq_model.summary()

* ***A Functional API Model***

In [ ]:
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)
model = Model(input_tensor, output_tensor)
model.summary()

The only part that may seem a bit magical at this point is instantiating a `Model` object
using only an input tensor and an output tensor. Behind the scenes, Keras retrieves
every layer involved in going from `input_tensor` to `output_tensor`, bringing them
together into a graph-like data structure—a `Model`. Of course, the reason it works is
that `output_tensor` was obtained by repeatedly transforming `input_tensor`.

When it comes to compiling, training, or evaluating such an instance of `Model`, the
API is the same as that of Sequential:

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics =['acc'])
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))
model.fit(x_train, y_train, epochs=10, batch_size=128)

In [ ]:
score = model.evaluate(x_train, y_train, verbose=2)

### 2. Build a multiple inputs model using functional API
We will build a question-answering model using functional API. The model has two inputs: a natural-language question
and a text snippet (such as a news article) providing information to be used for answering the question. The model merge two input branches using a `concatenate` layer, after that model produce an answer via a softmax over some predefined vocabulary.

First, we build up model using funcational API.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(100,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(100,),dtype='int32',name='question')
embedded_question = layers.Embedding(question_vocabulary_size,32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question],axis=-1)
answer = layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Then, we generate some dummy inputs (text, questions, answers) using `numpy` random number generation fuction.

In [ ]:
import numpy as np
num_samples = 1000
max_length = 100
text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(0, answer_vocabulary_size, size=(num_samples, answer_vocabulary_size))

In [ ]:
print(answers.shape)

Finally, we train the model using the above dummy inputs.

In [ ]:
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes = True, to_file='model.png')